#preprossing a video
in 4 step :
1. extact frames of video
2. preprocessing of frames
3. convert frame to video

In [ ]:
!pip install open_cv
!pip install ultralytics

In [ ]:
import cv2
import numpy as np
import os
import glob
import re

In [ ]:
# Define the image processing function
def process_and_enhance_image(image_path):
    # Read the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    stretched_image = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    processed_image = stretched_image

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(50 , 50))
    processed_image = clahe.apply(image)

    # Apply Laplacian filter for whole picture sharpening
    laplacian = cv2.Laplacian(processed_image, cv2.CV_64F)
    # Convert Laplacian to the same type as processed_image
    laplacian = np.uint8(np.absolute(laplacian))
    sharpened_image = cv2.addWeighted(processed_image, 1.5, laplacian, -0.5, 0)
    final_image = sharpened_image

    return final_image

In [ ]:
def enhance_low_light_image(image_path):
    # Read the image from the given path
    image = cv2.imread(image_path)

    # Convert the image to the LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)

    # Split the LAB image to different channels
    l, a, b = cv2.split(lab_image)

    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) to the L-channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)

    # Merge the CLAHE enhanced L-channel with the a and b channels
    merged_lab = cv2.merge((cl, a, b))

    # Convert the LAB image back to BGR color space
    enhanced_image = cv2.cvtColor(merged_lab, cv2.COLOR_Lab2BGR)

    # Apply Gamma Correction to reduce the darkness
    gamma = 1.2  # Change the value to adjust brightness
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    gamma_corrected_image = cv2.LUT(enhanced_image, table)

    # Apply a bilateral filter to reduce noise while keeping edges sharp
    final_image = cv2.bilateralFilter(gamma_corrected_image, d=9, sigmaColor=75, sigmaSpace=75)

    # final_image = cv2.cvtColor(final_image, cv2.COLOR_BGR2gray)

    # Save the final image to the specified output path
    cv2.imwrite('v1.jpg' , final_image)

    # Read the image in grayscale
    final_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Apply Laplacian filter for whole picture sharpening
    laplacian = cv2.Laplacian(final_image, cv2.CV_64F)
    # Convert Laplacian to the same type as processed_image
    laplacian = np.uint8(np.absolute(laplacian))
    sharpened_image = cv2.addWeighted(final_image , 1.5, laplacian, -0.5, 0)
    final_image = sharpened_image

    return final_image

In [ ]:
# Function to process all images in a directory
def process_images_in_directory(input_folder, output_folder , night_mode = False):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get all image paths from the input folder
    image_paths = glob.glob(os.path.join(input_folder, '*.jpg'))

    # Process each image
    for image_path in image_paths:
        # Process and enhance the image
        if night_mode:
            final_image = enhance_low_light_image(image_path)
        else:
            final_image = process_and_enhance_image(image_path)

        # Save the final image
        base_name = os.path.basename(image_path)
        output_path = os.path.join(output_folder, f'{base_name}')
        cv2.imwrite(output_path, final_image)

    # Print the status
    print(f"Image {base_name} has been processed and saved as {output_path}")

In [ ]:
def images_to_video(input_folder, output_video_path, fps=20):
    # Helper function to extract frame number from filename
    def frame_number(file_path):
        # Extract the base name and the number from the filename
        basename = os.path.basename(file_path)
        number = re.search(r'\d+', basename)
        return int(number.group()) if number else -1

    # Get all the image file paths and sort them by frame number
    img_paths = sorted(glob.glob(f"{input_folder}/*.jpg"), key=frame_number)

    # Read the first image to get the dimensions
    frame = cv2.imread(img_paths[0])
    height, width, layers = frame.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'XVID'
    video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    for img_path in img_paths:
        frame = cv2.imread(img_path)
        video.write(frame)  # Write out frame to video

    # Release everything when job is finished
    video.release()
    cv2.destroyAllWindows()

In [ ]:
# Function to extract frames with zero-padded filenames
def extract_frames(video_path, output_folder):
    # Check if output folder exists, if not, create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Load the video
    video = cv2.VideoCapture(video_path)

    # Get total number of frames in the video
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the number of digits needed for zero-padding
    padding = len(str(total_frames))

    # Initialize frame count
    count = 0

    # Loop through frames
    while video.isOpened():
        # Read video frame
        success, frame = video.read()

        # If successful, save frame as JPEG file
        if success:
            # Define the filename with zero-padding
            filename = f"{output_folder}/frame_{'{:0{padding}d}'.format(count, padding=padding)}.jpg"
            # Save the frame
            cv2.imwrite(filename, frame)
            count += 1
        else:
            break

        # Release the video capture object
    video.release()
    cv2.destroyAllWindows()

In [ ]:
video_input_path = '/content/Carsdrivingatnight_3.mp4'  # video path
output_directory = '/content/frames'  # output directory
extract_frames(video_input_path, output_directory)

In [ ]:
input_directory = '/content/frames'  # input directory path
output_directory = '/content/processed'  # output directory path
process_images_in_directory(input_directory, output_directory)

In [ ]:
input_directory = '/content/processed'  #   input directory path
output_video = '/content/output_video2.mp4'  # output video path
images_to_video(input_directory, output_video)